### Summary:

The provided code performs various tasks related to fantasy baseball roster selection on the Fan Duel platform. Here's a summary of its purpose:

1. Importing packages: The code imports pandas, pybaseball, itertools, and numpy packages for data manipulation, baseball data retrieval, and mathematical operations.

2. Gather relevant player data: The code reads a CSV file obtained from statcast.

3. Defining the "outs_made" function: This function calculates the number of outs made based on the result of a play in a play-by-play data frame.

4. Uploading and cleaning Fan Duel data: The code reads a CSV file containing Fan Duel data, removes rows with injury indicators, and drops unnecessary columns.

5. Gathering more basic player stats: The code downloads Lahman's Baseball Database and retrieves fielding, pitching, batting, and people (player information) data.

6. Gathering relevant player information: The code matches players from the Fan Duel data with their corresponding player IDs and retrieves relevant player information.

7. Processing and filtering player data: The code processes and filters the Fan Duel data based on specific criteria, such as minimum plate appearances for batters and outs made for pitchers.

8. Calculating expected values: The code calculates the expected value per start for both pitchers and non-pitchers based on various performance metrics.

9. Seperating players by position: The code separates non-pitchers into different data frames based on their Fan Duel roster positions.

10. Creating the optimization problem: The code sets up an optimization problem using the PuLP package to maximize the expected value per start while considering constraints on positions, salary, and the total number of players.

11. Solving the linear programming problem: The code solves the optimization problem to select the best players for the fantasy baseball lineup.

12. Printing the results: The code prints the selected players, their average points, the maximum sum of average points, and the salary spent.

13. Printing the chosen players, Fan Duel fantasy salary, and their expected fantasy value: The code prints specific information about the selected players, such as their names, salaries, and expected fantasy values.

Overall, this code performs data manipulation, data retrieval, filtering, optimization, and result printing to assist with fantasy baseball lineup selection.

In [1]:
#PACKAGES
import pandas as pd
import pybaseball as pyb
from pybaseball import statcast
import itertools
import numpy as np

In [ ]:
import pybaseball
# Retrieve the statcast data for a specific time period
pybaseball.cache.enable()
start_date = '2023-06-07'
end_date = '2023-12-31'
data_frame_update = statcast(start_date, end_date)
# Concatenate the two DataFrames 
data_frame = pd.concat([data_frame_update, data_frame])

In [3]:
#download presaved data frame
data_frame = pd.read_csv('/Users/tmpolstra/Library/CloudStorage/Dropbox/Baseball/Fantasy Sports/play_by_play')

In [ ]:
data_frame.to_csv('play_by_play', index=False)

In [4]:
#This function allows one to compute the number of outs made from the result of a particular play from the play_by_play
#data_frame.
def outs_made(df):
    index = df.index[0]
    one_out_plays = ['field_out', 'strikeout', 'sac_bunt', 'force_out',
                    'sac_fly', 'fielders_choice', 'fielders_choice_out','caught_stealing_2b',
                    'pickoff_caught_stealing_2b', 'pickoff_1b', 'pickoff_3b', 'other_out', 'caught_stealing_home',
                    'caught_stealing_3b', 'pickoff_caught_stealing_home', 'pickoff_caught_stealing_3b',
                    'pickoff_2b']
    two_out_plays = ['grounded_into_double_play', 'double_play', 'strikeout_double_play',
                     'sac_fly_double_play', 'sac_bunt_double_play', 'runner_double_play']
    three_out_plays = ['triple_play']
    if df['events'][index] in one_out_plays:
        return 1
    if df['events'][index] in two_out_plays:
        return 2
    if df['events'][index] in three_out_plays:
        return 3
    else:
        return 0

In [8]:
#Upload csv file from Fan Duel and clean the data
fan_duel_data = pd.read_csv('/Users/tmpolstra/Downloads/FanDuel-MLB-2023 ET-06 ET-09 ET-91203-players-list (1).csv')
fan_duel_data = fan_duel_data.drop(fan_duel_data[fan_duel_data['Injury Indicator'].notna()].index)
fan_duel_data = fan_duel_data.drop('Injury Indicator', axis=1)
fan_duel_data = fan_duel_data.drop('Injury Details', axis=1)
fan_duel_data = fan_duel_data.drop('Tier', axis=1)
fan_duel_data = fan_duel_data.drop('FPPG', axis=1)
fan_duel_data = fan_duel_data.drop('Played', axis=1)
fan_duel_data = fan_duel_data.drop('Nickname', axis=1)

#Gather basic player stats
from pybaseball.lahman import *
download_lahman()

from pybaseball import playerid_reverse_lookup

# fielding stats by year 
fielding = fielding()
pitching = pitching()
batting = batting()
people = people()


#Gather relevant player information
relevant_players_playerID = []
relevant_players = pd.DataFrame()
fan_duel_data['Last Name'] = fan_duel_data['Last Name'].str.replace(' Jr.$', '', regex=True)
fan_duel_data['playerID'] = ''
for i in fan_duel_data.index:
    first_name = fan_duel_data.loc[i, 'First Name']
    last_name = fan_duel_data.loc[i, 'Last Name']
    player_data = people[(people['nameFirst'].str.lower() == first_name.lower()) & (people['nameLast'].str.lower() == last_name.lower())]
    relevant_players_playerID+=[ID for ID in player_data['playerID']]
    fan_duel_data.loc[i,'playerID'] = relevant_players_playerID[-1]
    relevant_players = pd.concat([relevant_players, player_data], ignore_index=True)

people = people[people['playerID'].isin(relevant_players_playerID)]
fielding = fielding[fielding['playerID'].isin(relevant_players_playerID)]
batting = batting[batting['playerID'].isin(relevant_players_playerID)]
pitching = pitching[pitching['playerID'].isin(relevant_players_playerID)]

fan_duel_data['mlbam'] = 0
fan_duel_data['idFG'] = 0
for i in fan_duel_data.index:
    playerID = fan_duel_data.loc[i, 'playerID']
    player_data = playerid_reverse_lookup([playerID], key_type='bbref')  # Pass playerID as a list
    
    if not player_data.empty:
        mlbam = player_data.loc[0, 'key_mlbam']
        fan_duel_data.loc[i, 'mlbam'] = mlbam
        idFG = player_data.loc[0, 'key_fangraphs']
        fan_duel_data.loc[i, 'idFG'] = idFG
fan_duel_data = fan_duel_data[fan_duel_data['mlbam'] != 0]
fan_duel_data = fan_duel_data[fan_duel_data['idFG'] != 0]


#Seperate pitchers from non-pitchers
fan_duel_P = fan_duel_data[fan_duel_data['Position'] == 'P'].copy()
fan_duel_non_pitchers = fan_duel_data[~(fan_duel_data['Position'] == 'P')].copy()
fan_duel_non_pitchers = fan_duel_non_pitchers.drop('Probable Pitcher', axis=1)

#Only examine batters with at least 220 at plate appearances and pitchers that have achieved at least 220 outs.
for ID in relevant_players_playerID:
    at_bats = sum(batting[batting['playerID'] == ID]['AB'])
    if at_bats < 220:
        fan_duel_non_pitchers.drop(fan_duel_non_pitchers[fan_duel_non_pitchers['playerID'] == ID].index, inplace=True)

for ID in relevant_players_playerID:
    batters_retired = sum(pitching[pitching['playerID'] == ID]['IPouts'])
    if batters_retired < 220:
        fan_duel_P.drop(fan_duel_P[fan_duel_P['playerID'] == ID].index, inplace=True)
        
        
fan_duel_P=fan_duel_P[fan_duel_P['Probable Pitcher']=='Yes']
fan_duel_P = fan_duel_P.drop('Batting Order', axis=1)
fan_duel_P = fan_duel_P.drop('Roster Position', axis=1)
fan_duel_P = fan_duel_P.drop('Position', axis=1)
fan_duel_P = fan_duel_P.drop('Probable Pitcher', axis=1)

#Gather relevant pitching data
fan_duel_P['Games_Started'] = 0
for i in fan_duel_P.index:
    ID = fan_duel_P.at[i, 'playerID']
    pitching_stats = pitching[pitching['playerID'] == ID]
    games_started = sum(pitching_stats['GS'])
    fan_duel_P.at[i, 'Games_Started'] = games_started

fan_duel_P['Quality_Starts'] = 0
for i in fan_duel_P.index:
    ID = fan_duel_P['mlbam'][i]
    pitcher_data = data_frame[data_frame['pitcher.1'] == ID]
    
    dates_pitched = pitcher_data['game_date'].unique()
    quality_starts = 0
    for date in dates_pitched:
        pitcher_performance = pitcher_data[pitcher_data['game_date'] == date]
        pitcher_performance_order = pitcher_performance[::-1].copy()
        pitcher_performance_order = pitcher_performance_order.reset_index(drop=True)
        if pitcher_performance_order['inning'].iloc[0] != 1:
            continue
        if (pitcher_performance_order['inning'].iloc[-1]+pitcher_performance_order['outs_when_up'].iloc[-1]*.1>= 5.2) and (pitcher_performance_order['post_bat_score'].iloc[-1]<=3):
            quality_starts += 1
    fan_duel_P.loc[i,'Quality_Starts'] = quality_starts
    
fan_duel_P['average_outs_per_start'] = 0
for i in fan_duel_P.index:
    ID = fan_duel_P['mlbam'][i]
    pitcher_data = data_frame[data_frame['pitcher.1'] == ID]
    
    dates_pitched = pitcher_data['game_date'].unique()
    
    outs_list = []
    for date in dates_pitched:
        pitcher_performance = pitcher_data[pitcher_data['game_date'] == date]
        pitcher_performance_order = pitcher_performance[::-1].copy()
        pitcher_performance_order = pitcher_performance_order.reset_index(drop=True)
        if pitcher_performance_order['inning'].iloc[0] != 1:
            continue
        inning = pitcher_performance_order['inning'].iloc[-1]
        outs = 3*(inning-1)+ pitcher_performance_order['outs_when_up'].iloc[-1]
        outs_from_last_play = outs_made(pitcher_performance_order)
        outs += outs_from_last_play
        outs_list.append(outs)
    
    average_outs_per_game = sum(outs_list) / len(outs_list)
    fan_duel_P.loc[i, 'average_outs_per_start'] = average_outs_per_game

fan_duel_data['expected_value_per_start'] = 0
fan_duel_P['expected_value_per_start'] = 0
for i in fan_duel_P.index:
    playerID = fan_duel_P.at[i, 'playerID']
    ER = sum(pitching[pitching['playerID'] == playerID]['ER'])
    IP = sum(pitching[pitching['playerID'] == playerID]['IPouts'])
    ER_per_out = ER/IP
    SO = sum(pitching[pitching['playerID'] == playerID]['SO'])
    Strikeout_ratio = SO/IP
    Wins = sum(pitching[pitching['playerID'] == playerID]['W'])
    QStarts = fan_duel_P.at[i, 'Quality_Starts']
    Starts = fan_duel_P.at[i, 'Games_Started']
    Win_percentage = Wins/Starts
    expected_value = (-ER_per_out+3*Strikeout_ratio+1)*fan_duel_P['average_outs_per_start'][i] + 4* QStarts/Starts + 6*Wins/Starts
    fan_duel_P.at[i, 'expected_value_per_start'] = expected_value
    fan_duel_data.at[i, 'expected_value_per_start'] = expected_value

#Gather relevant non-pitcher data
fan_duel_non_pitchers['expected_value_per_bat'] = 0
for i in fan_duel_non_pitchers.index:
    playerID = fan_duel_non_pitchers['playerID'][i]
    total_value = 3*(sum(batting[batting['playerID'] == playerID]['H'])) + 3*(sum(batting[batting['playerID'] == playerID]['2B']))+6*(sum(batting[batting['playerID'] == playerID]['3B'])) + 9*(sum(batting[batting['playerID'] == playerID]['HR'])) + 3*(sum(batting[batting['playerID'] == playerID]['HBP'])) + 3*(sum(batting[batting['playerID'] == playerID]['BB'])) + 3.2*(sum(batting[batting['playerID'] == playerID]['R'])) + 3.5*(sum(batting[batting['playerID'] == playerID]['RBI'])) + 6*(sum(batting[batting['playerID'] == playerID]['SB']))
    expected_value = total_value / (sum(batting[batting['playerID'] == playerID]['AB']))
    fan_duel_non_pitchers.loc[i, 'expected_value_per_bat'] = expected_value
    
    
    
fan_duel_non_pitchers['expected_number_of_bats'] = 0
for i, row in fan_duel_non_pitchers.iterrows():
    mlbam = row['mlbam']
    batter_data_frame = data_frame[data_frame['batter'] == mlbam]
    expected_number_of_bats = batter_data_frame.groupby('game_date')['at_bat_number'].nunique().mean()
    fan_duel_non_pitchers.at[i, 'expected_number_of_bats'] = expected_number_of_bats

fan_duel_non_pitchers['expected_value_per_start'] = 0
for i in fan_duel_non_pitchers.index:
    playerID = fan_duel_non_pitchers['playerID'][i]
    expected_value_per_start = fan_duel_non_pitchers['expected_value_per_bat'][i]*fan_duel_non_pitchers['expected_number_of_bats'][i]
    fan_duel_non_pitchers.loc[i, 'expected_value_per_start'] = expected_value_per_start
    fan_duel_data.loc[i, 'expected_value_per_start'] = expected_value_per_start

#Seperate non-pitchers by their Fan Duel roster position
fan_duel_C_1B = fan_duel_non_pitchers[(fan_duel_non_pitchers['Roster Position'].str.contains('1B')) | (fan_duel_non_pitchers['Position'].str.contains('C'))].copy()
fan_duel_2B = fan_duel_non_pitchers[fan_duel_non_pitchers['Roster Position'].str.contains('2B')].copy()
fan_duel_3B = fan_duel_non_pitchers[fan_duel_non_pitchers['Roster Position'].str.contains('3B')].copy()
fan_duel_SS = fan_duel_non_pitchers[fan_duel_non_pitchers['Roster Position'].str.contains('SS')].copy()
fan_duel_UTIL = fan_duel_non_pitchers[fan_duel_non_pitchers['Roster Position'].str.contains('UTIL')].copy()
fan_duel_OF = fan_duel_non_pitchers[fan_duel_non_pitchers['Roster Position'].str.contains('OF')].copy()


relevant_player_data_frame = pd.concat([fan_duel_P, fan_duel_non_pitchers], ignore_index=False)



In [19]:
fan_duel_P

,Id,First Name,Last Name,Salary,Game,Team,Opponent,playerID,mlbam,idFG,Games_Started,Quality_Starts,average_outs_per_start,expected_value_per_start
7,91203-102335,Cristian,Javier,10800,HOU@CLE,HOU,CLE,javiecr01,664299,17606,44,47,15.500000,40.003286
9,91203-16956,Gerrit,Cole,10500,BOS@NYY,NYY,BOS,colege01,543037,13125,267,133,17.611429,41.015507
16,91203-39041,Marcus,Stroman,10200,CHC@SF,CHC,SF,stromma01,573186,13431,198,84,15.875000,30.697995
20,91203-82440,Shohei,Ohtani,10000,SEA@LAA,LAA,SEA,ohtansh01,660271,19755,63,54,15.696203,39.866763
26,91203-79091,Dylan,Cease,9800,MIA@CWS,CWS,MIA,ceasedy01,656302,18525,90,75,15.071429,36.931086
32,91203-145922,Tyler,Wells,9700,KC@BAL,BAL,KC,wellsty01,669330,20000,23,19,13.275000,28.511614
36,91203-17064,Sonny,Gray,9600,MIN@TOR,MIN,TOR,grayso01,543243,12768,238,88,15.030534,31.294848
40,91203-16931,Yu,Darvish,9500,SD@COL,SD,COL,darviyu01,506433,13074,242,88,16.839695,38.700226
63,91203-53607,Anthony,DeSclafani,8800,CHC@SF,SF,CHC,desclan01,543101,13050,151,65,15.068376,29.901010
71,91203-60646,Ben,Lively,8700,CIN@STL,CIN,STL,livelbe01,594902,14932,20,5,15.666667,25.525926


In [20]:
fan_duel_P[['First Name','Last Name', 'Salary', 'expected_value_per_start']]

,First Name,Last Name,Salary,expected_value_per_start
7,Cristian,Javier,10800,40.003286
9,Gerrit,Cole,10500,41.015507
16,Marcus,Stroman,10200,30.697995
20,Shohei,Ohtani,10000,39.866763
26,Dylan,Cease,9800,36.931086
32,Tyler,Wells,9700,28.511614
36,Sonny,Gray,9600,31.294848
40,Yu,Darvish,9500,38.700226
63,Anthony,DeSclafani,8800,29.901010
71,Ben,Lively,8700,25.525926


In [30]:
fan_duel_non_pitchers[['First Name','Last Name', 'Salary', 'expected_value_per_start']]

,First Name,Last Name,Salary,expected_value_per_start
623,Ronald,Acuna,4600,15.657922
624,Fernando,Tatis,4500,16.145104
625,Freddie,Freeman,4400,13.712730
626,Mookie,Betts,4300,14.968044
628,J.D.,Martinez,4200,14.968044
...,...,...,...,...
1090,Michael,Perez,2000,5.873247
1094,Dylan,Moore,2000,9.563545
1095,Luke,Williams,2000,4.996373
1101,Jordan,Luplow,2000,8.433957


In [28]:
pre_chosen_players = [7]  # Specify the indices of pre-chosen players
#pre_chosen_players = []
import pulp

# Create the optimization problem
prob = pulp.LpProblem("FantasyBaseball", pulp.LpMaximize)

# Define decision variables
players = list(relevant_player_data_frame.index)
x = pulp.LpVariable.dict("x", players, cat='Binary')

# Set up the objective function
objective = pulp.lpSum(relevant_player_data_frame.loc[player, 'expected_value_per_start'] * x[player] for player in players)
prob += objective

# Predetermined player constraints
for player in pre_chosen_players:
    prob += x[player] == 1

# Position constraints
pitcher_constraint = pulp.lpSum(x[player] for player in fan_duel_P.index) == 1
prob += pitcher_constraint

infield_constraints = []
for position_df in [fan_duel_C_1B, fan_duel_2B, fan_duel_3B, fan_duel_SS]:
    constraint = pulp.lpSum(x[player] for player in position_df.index) == 1
    infield_constraints.append(constraint)
    prob += constraint

outfield_constraint = pulp.lpSum(x[player] for player in fan_duel_OF.index) == 3
prob += outfield_constraint

# Total players constraint (select exactly 9 players)
total_players_constraint = pulp.lpSum(x[player] for player in players) == 9
prob += total_players_constraint

# Salary constraint
salary_limit = 35000
salary_constraint = pulp.lpSum(relevant_player_data_frame.loc[player, 'Salary'] * x[player] for player in players)
prob.addConstraint(salary_constraint <= salary_limit, name="Salary Constraint")

# Solve the linear programming problem
prob.solve()

# Retrieve the selected players and their corresponding average points
selected_players=[]
selected_players = [player for player in players if x[player].varValue == 1]
selected_average_points = [relevant_player_data_frame.loc[player, 'expected_value_per_start'] for player in selected_players]

# Verify and adjust salary limit if necessary
#while sum(relevant_player_data_frame.loc[selected_players, 'Salary']) > salary_limit or len(selected_players)!=9:
    #salary_limit -= 100
    #salary_constraint_bound = salary_constraint <= salary_limit
    #prob.constraints["Salary Constraint"] = salary_constraint_bound
    #prob.solve()
    #selected_players = [player for player in players if x[player].varValue == 1]

# Print the results
print("Selected Players:")
for player, avg_points in zip(selected_players, selected_average_points):
    print(f"Player: {player}, Average Points: {avg_points}")

# Print the maximum sum of average points
max_sum_avg_points = pulp.value(prob.objective)
print('')
print(f"Maximum Sum of Average Points: {max_sum_avg_points}")

players_selected = relevant_player_data_frame.loc[selected_players]

print('')
print(f"Salary Spent: {sum(players_selected['Salary'])}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/tmpolstra/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/y9/h1gp0h4s6hj68gv7rdh9l6tm0000gq/T/604be035860e41d6bb6813ec3aa97abb-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y9/h1gp0h4s6hj68gv7rdh9l6tm0000gq/T/604be035860e41d6bb6813ec3aa97abb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 1972 RHS
At line 1982 BOUNDS
At line 2295 ENDATA
Problem MODEL has 9 rows, 312 columns and 1021 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Selected Players:
Player: 7, Average Points: 40.00328587075575
Player: 629, Average Points: 15.922802193460708
Player: 647, Average Points

In [29]:
#Print chosen players, Fan Duel fantasy salary, and their expected fantansy value
players_selected[['First Name', 'Last Name', 'Salary','expected_value_per_start']]

,First Name,Last Name,Salary,expected_value_per_start
7,Cristian,Javier,10800,40.003286
629,Juan,Soto,4100,15.922802
647,Mike,Trout,3600,16.466481
672,George,Springer,3300,14.046516
702,Alex,Bregman,3100,13.831068
716,Trea,Turner,3000,14.144872
724,Ozzie,Albies,3000,12.121749
971,Cavan,Biggio,2300,10.833509
